In [1]:
import pandas as pd
import zipfile
import io
import numpy as np

from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.callbacks import ModelCheckpoint, TensorBoard


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

Using TensorFlow backend.


In [2]:
def load_dataset(filename):
    df = pd.read_excel(filename)
    intent = df["intent"]
    unique_intent = list(set(intent))
    text = list(df["text"])
    return (intent, unique_intent, text, df)


def load_word_vectors(filepath,vocab,outfile):
    with zipfile.ZipFile(filepath) as zfile:
        with open(outfile,"w") as sub_vector:
            for finfo in zfile.infolist():
                ifile = zfile.open(finfo)
                textStream = io.TextIOWrapper(ifile, encoding='utf-8')
                #n, d = map(int, textStream.readline().split())
                #print("Number of tokens = "+str(n))
                data = {}
                lines=[]
                for line in textStream:
                    tokens = line.rstrip().split(' ')
                    word=tokens[0]
                    if(word in vocab):
                        lines.append(line)
                        data[word] = np.asarray(tokens[1:], dtype='float32')
                sub_vector.writelines(lines);
        return data

        
def build_embedding_matrix(tok,vectors,embedding_size):
    vocab_size=len(tok.word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_size))
    for word, i in tok.word_index.items():
        embedding_vector = vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


def get_vocab(sentences):
    text_words=list(map(lambda x: text_to_word_sequence(x,lower=False),text))
    vocab=set()
    for words in text_words:
        vocab.update(words)
    return vocab


def prepare_training_data(text,intent,max_length,embedding_size):
    tok = Tokenizer(lower=False)
    tok.fit_on_texts(text)
    encoded_docs = tok.texts_to_sequences(text)
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    embedding_matrix = build_embedding_matrix(tok,glove_vectors,embedding_size)
    vocab_size = len(tok.word_index) + 1
    le = LabelEncoder()
    labels_encoded=le.fit_transform(intent)
    ohe = OneHotEncoder(sparse=False,categories='auto')
    output_one_hot = ohe.fit_transform(labels_encoded.reshape(-1, 1))
    train_X, val_X, train_Y, val_Y = train_test_split(padded_docs, output_one_hot, shuffle = True, test_size = 0.2)
    return (train_X, val_X, train_Y, val_Y, vocab_size, embedding_matrix)


def build_model(embedding_matrix,max_length, label_count, use_pre_trained_vectors=True):
    model = Sequential()
    if use_pre_trained_vectors:
        e = Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                      input_length=max_length, trainable=False)
    else:
        e = Embedding(vocab_size, embedding_size, input_length=max_length)
    
    model.add(e)
    model.add(Bidirectional(GRU(256)))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(label_count, activation = "softmax"))
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return model

In [3]:
intent, unique_intent, text, df = load_dataset("./data/cogcomp_org_Experimental_Data_for_Question_Classification.xlsx")

In [4]:
vocab=get_vocab(text)

In [5]:
label_count=len(unique_intent)

In [6]:
glove_vectors=load_word_vectors("./data/pre-trained-vectors/crawl-300d-2M-subword.zip",vocab,'./data/pre-trained-vectors/crawl-300d-2M.subset_for_vocab.vec')

In [7]:
max_length=20
embedding_size = 300

train_X, val_X, train_Y, val_Y, vocab_size, embedding_matrix = prepare_training_data(text,intent,max_length,embedding_size)

In [8]:
model = build_model(embedding_matrix,max_length,label_count,use_pre_trained_vectors=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:

#model.summary()

In [10]:
filepath="./data/model/pre-trained_vectors/model-epoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5"
log_path="./logs/pre-trained/"
tboard_callback=TensorBoard(log_dir=log_path)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint,tboard_callback]

In [11]:
model.fit(train_X, train_Y, epochs = 500, batch_size = 32, validation_data = (val_X, val_Y),callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Train on 4361 samples, validate on 1091 samples
Epoch 1/500
4361/4361 [==============================] - 8s 2ms/step - loss: 3.4447 - acc: 0.1903 - val_loss: 2.6264 - val_acc: 0.3831

Epoch 00001: val_acc improved from -inf to 0.38313, saving model to ./data/model/pre-trained_vectors/model-epoch-01-val_acc-0.38.hdf5
Epoch 2/500
4361/4361 [==============================] - 7s 2ms/step - loss: 2.6575 - acc: 0.3671 - val_loss: 2.1683 - val_acc: 0.4528

Epoch 00002: val_acc improved from 0.38313 to 0.45280, saving model to ./data/model/pre-trained_vectors/model-epoch-02-val_acc-0.45.hdf5
Epoch 3/500
4361/4361 [==============================] - 6s 1ms/step - loss: 2.3698 - acc: 0.4231 - val_loss: 2.1051 - val_acc: 0.4647

Epoch 00003: val_acc improved from 0.45280 to 0.46471, saving model to ./data/model/pre-trained_vectors/model-epoch-03-val_acc-0.46.hdf5
Epoch 4/500
4361/4361 [==============================] - 6s 1ms/step - loss: 2.1883 - ac

4361/4361 [==============================] - 6s 1ms/step - loss: 0.7175 - acc: 0.8019 - val_loss: 1.0734 - val_acc: 0.7397

Epoch 00034: val_acc did not improve from 0.75435
Epoch 35/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.6844 - acc: 0.8136 - val_loss: 1.0449 - val_acc: 0.7461

Epoch 00035: val_acc did not improve from 0.75435
Epoch 36/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.6525 - acc: 0.8234 - val_loss: 1.0107 - val_acc: 0.7764

Epoch 00036: val_acc improved from 0.75435 to 0.77635, saving model to ./data/model/pre-trained_vectors/model-epoch-36-val_acc-0.78.hdf5
Epoch 37/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.6756 - acc: 0.8216 - val_loss: 1.0108 - val_acc: 0.7644

Epoch 00037: val_acc did not improve from 0.77635
Epoch 38/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.6354 - acc: 0.8241 - val_loss: 1.0091 - val_acc: 0.7736

Epoch 00038: val_acc did not improve from

4361/4361 [==============================] - 6s 1ms/step - loss: 0.3423 - acc: 0.9044 - val_loss: 1.0915 - val_acc: 0.7965

Epoch 00075: val_acc did not improve from 0.80293
Epoch 76/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.3098 - acc: 0.9101 - val_loss: 1.0414 - val_acc: 0.7938

Epoch 00076: val_acc did not improve from 0.80293
Epoch 77/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.3033 - acc: 0.9117 - val_loss: 1.0758 - val_acc: 0.7965

Epoch 00077: val_acc did not improve from 0.80293
Epoch 78/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.3182 - acc: 0.9106 - val_loss: 1.1518 - val_acc: 0.7965

Epoch 00078: val_acc did not improve from 0.80293
Epoch 79/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.2950 - acc: 0.9149 - val_loss: 1.1397 - val_acc: 0.7929

Epoch 00079: val_acc did not improve from 0.80293
Epoch 80/500
4361/4361 [==============================] - 6s 1ms/step - loss: 

4361/4361 [==============================] - 6s 1ms/step - loss: 0.2352 - acc: 0.9365 - val_loss: 1.1192 - val_acc: 0.8057

Epoch 00117: val_acc did not improve from 0.81760
Epoch 118/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.2398 - acc: 0.9337 - val_loss: 1.1307 - val_acc: 0.8084

Epoch 00118: val_acc did not improve from 0.81760
Epoch 119/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.2662 - acc: 0.9271 - val_loss: 1.0975 - val_acc: 0.8084

Epoch 00119: val_acc did not improve from 0.81760
Epoch 120/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.2171 - acc: 0.9415 - val_loss: 1.0848 - val_acc: 0.8038

Epoch 00120: val_acc did not improve from 0.81760
Epoch 121/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.2242 - acc: 0.9337 - val_loss: 1.0380 - val_acc: 0.8148

Epoch 00121: val_acc did not improve from 0.81760
Epoch 122/500
4361/4361 [==============================] - 6s 1ms/step - l

4361/4361 [==============================] - 6s 1ms/step - loss: 0.2241 - acc: 0.9383 - val_loss: 1.2235 - val_acc: 0.8103

Epoch 00160: val_acc did not improve from 0.82218
Epoch 161/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1820 - acc: 0.9491 - val_loss: 1.1901 - val_acc: 0.8029

Epoch 00161: val_acc did not improve from 0.82218
Epoch 162/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1698 - acc: 0.9512 - val_loss: 1.2348 - val_acc: 0.8066

Epoch 00162: val_acc did not improve from 0.82218
Epoch 163/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1939 - acc: 0.9427 - val_loss: 1.1936 - val_acc: 0.8176

Epoch 00163: val_acc did not improve from 0.82218
Epoch 164/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.2080 - acc: 0.9408 - val_loss: 1.2501 - val_acc: 0.8048

Epoch 00164: val_acc did not improve from 0.82218
Epoch 165/500
4361/4361 [==============================] - 6s 1ms/step - l

...


Epoch 00289: val_acc did not improve from 0.82493
Epoch 290/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1438 - acc: 0.9590 - val_loss: 1.2857 - val_acc: 0.8038

Epoch 00290: val_acc did not improve from 0.82493
Epoch 291/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1446 - acc: 0.9594 - val_loss: 1.3617 - val_acc: 0.8093

Epoch 00291: val_acc did not improve from 0.82493
Epoch 292/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1536 - acc: 0.9548 - val_loss: 1.3816 - val_acc: 0.8167

Epoch 00292: val_acc did not improve from 0.82493
Epoch 293/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1452 - acc: 0.9601 - val_loss: 1.3893 - val_acc: 0.8093

Epoch 00293: val_acc did not improve from 0.82493
Epoch 294/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1526 - acc: 0.9629 - val_loss: 1.3684 - val_acc: 0.8112

Epoch 00294: val_acc did not improve from 0.82493
Epoch 295

4361/4361 [==============================] - 6s 1ms/step - loss: 0.1197 - acc: 0.9681 - val_loss: 1.4385 - val_acc: 0.8103

Epoch 00333: val_acc did not improve from 0.83043
Epoch 334/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1455 - acc: 0.9580 - val_loss: 1.3422 - val_acc: 0.8148

Epoch 00334: val_acc did not improve from 0.83043
Epoch 335/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1390 - acc: 0.9596 - val_loss: 1.3038 - val_acc: 0.8203

Epoch 00335: val_acc did not improve from 0.83043
Epoch 336/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1248 - acc: 0.9635 - val_loss: 1.3633 - val_acc: 0.8213

Epoch 00336: val_acc did not improve from 0.83043
Epoch 337/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1270 - acc: 0.9622 - val_loss: 1.3436 - val_acc: 0.8112

Epoch 00337: val_acc did not improve from 0.83043
Epoch 338/500
4361/4361 [==============================] - 6s 1ms/step - l


Epoch 00376: val_acc did not improve from 0.83043
Epoch 377/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1315 - acc: 0.9654 - val_loss: 1.3314 - val_acc: 0.8231

Epoch 00377: val_acc did not improve from 0.83043
Epoch 378/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1470 - acc: 0.9578 - val_loss: 1.2595 - val_acc: 0.8231

Epoch 00378: val_acc did not improve from 0.83043
Epoch 379/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1276 - acc: 0.9668 - val_loss: 1.3135 - val_acc: 0.8286

Epoch 00379: val_acc did not improve from 0.83043
Epoch 380/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1149 - acc: 0.9658 - val_loss: 1.2795 - val_acc: 0.8295

Epoch 00380: val_acc did not improve from 0.83043
Epoch 381/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1039 - acc: 0.9684 - val_loss: 1.3782 - val_acc: 0.8093

Epoch 00381: val_acc did not improve from 0.83043
Epoch 382

4361/4361 [==============================] - 6s 1ms/step - loss: 0.1186 - acc: 0.9651 - val_loss: 1.3524 - val_acc: 0.8304

Epoch 00419: val_acc did not improve from 0.83410
Epoch 420/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1140 - acc: 0.9665 - val_loss: 1.4137 - val_acc: 0.8203

Epoch 00420: val_acc did not improve from 0.83410
Epoch 421/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1403 - acc: 0.9622 - val_loss: 1.4776 - val_acc: 0.8093

Epoch 00421: val_acc did not improve from 0.83410
Epoch 422/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1110 - acc: 0.9651 - val_loss: 1.3879 - val_acc: 0.8194

Epoch 00422: val_acc did not improve from 0.83410
Epoch 423/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1102 - acc: 0.9700 - val_loss: 1.3972 - val_acc: 0.8213

Epoch 00423: val_acc did not improve from 0.83410
Epoch 424/500
4361/4361 [==============================] - 6s 1ms/step - l


Epoch 00461: val_acc did not improve from 0.83868
Epoch 462/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1314 - acc: 0.9674 - val_loss: 1.3167 - val_acc: 0.8240

Epoch 00462: val_acc did not improve from 0.83868
Epoch 463/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1101 - acc: 0.9697 - val_loss: 1.3577 - val_acc: 0.8203

Epoch 00463: val_acc did not improve from 0.83868
Epoch 464/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1139 - acc: 0.9690 - val_loss: 1.3044 - val_acc: 0.8203

Epoch 00464: val_acc did not improve from 0.83868
Epoch 465/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1217 - acc: 0.9679 - val_loss: 1.4161 - val_acc: 0.8249

Epoch 00465: val_acc did not improve from 0.83868
Epoch 466/500
4361/4361 [==============================] - 6s 1ms/step - loss: 0.1031 - acc: 0.9716 - val_loss: 1.3266 - val_acc: 0.8304

Epoch 00466: val_acc did not improve from 0.83868
Epoch 467